# AI Journalist

AI-Powered Article Generator: Collects and analyzes the latest news to craft high-quality articles autonomously.

AI 기반 자동 기사 생성 시스템: 최신 뉴스를 수집하고 분석하여 고품질의 기사를 작성합니다.

## What we want to know

- 기존 기사 데이터를 바탕으로 새로운 기사를 생성할 수 있는가
- Cost effective 하면서도 높은 품질의 기사를 생성할 수 있는가

## Pipeline

1. 필요한 라이브러리 설치 및 API 키 설정
2. 주제 설정 ("아이폰 16 출시 및 스펙")
3. AI(Claude)를 사용해 주제와 관련된 5개의 검색어 생성 (Claude 3 Haiku)
4. 생성된 검색어로 Google 뉴스 검색 수행 (SerpAPI)
5. 각 검색 결과에서 AI가 가장 관련성 높은 기사 선택 (Claude 3 Haiku)
6. 선택된 기사들의 본문 내용 추출
7. 추출된 내용을 바탕으로 AI가 한국어로 NYT 스타일의 기사 작성 (Claude 3.5 Sonnet)

## API Usage while development

Total cost: $0.3464 (461.29 South Korean won)

Claude 3 Haiku
- Input: $0.0049 (0.019 Million Input Tokens)
- Output: $0.0015 (0.001 Million Output Tokens)

Claude 3.5 Sonnet
- Input: $0.21 (0.071 Million Input Tokens) 
- Output: $0.13 (0.008 Million Output Tokens)

### Reference

- https://github.com/mshumer/ai-journalist
- YouTube [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

In [ ]:
%pip install anthropic
%pip install google-search-results
%pip install newspaper3k

## API keys

- https://console.anthropic.com/settings/keys
- https://serpapi.com/dashboard

In [1]:
import anthropic

ANTHROPIC_API_KEY = "YOUR_ANTHROPIC_API_KEY"
SERP_API_KEY = "YOUR_SERP_API_KEY"

## Topic

In [31]:
topic = "아이폰 16 출시 및 스펙"

print(topic)

아이폰 16 출시 및 스펙


## Search Terms

Topic을 바탕으로 5개의 검색어 생성

상대적으로 저렴한 Claude 3 Haiku 모델을 사용하여 검색어 생성

[Claude 모델 종류](https://docs.anthropic.com/en/docs/about-claude/models)

In [54]:
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

system_prompt = "You are a world-class journalist. Generate a list of 5 search terms to search for to research and write an article about the topic."
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": f"Please provide a list of 5 search terms related to '{topic}' for researching and writing an article. Respond with the search terms in a Python-parseable list, separated by commas. Ensure the search terms are specific to the topic and not too general. Ensure the search terms are written in English.\n\ne.g. ['search term 1', 'search term 2', 'search term 3', 'search term 4', 'search term 5']"
    }]
}]

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=200,
    temperature=0.5,
    system=system_prompt,
    messages=messages,
)

print(message.content[0].text)

['iPhone 16 release date', 'iPhone 16 specifications', 'iPhone 16 features', 'iPhone 16 design', 'iPhone 16 rumors']


### Parse search terms

String을 Python list 형식으로 변환

In [55]:
import re

response = message.content[0].text
list_pattern = r'\[(.*?)\]'
match = re.search(list_pattern, response)

if match:
    list_string = match.group(1)
    search_terms = [term.strip().strip("'") for term in list_string.split(',')]
    print(search_terms)
else:
    print("리스트를 찾을 수 없습니다.")

['iPhone 16 release date', 'iPhone 16 specifications', 'iPhone 16 features', 'iPhone 16 design', 'iPhone 16 rumors']


## Search for news articles

1. 각 검색어에 대해 뉴스 검색 수행 (Google Search)
2. 검색 결과에서 가장 관련성 높은 기사 선택 (Claude 3 Haiku)


In [66]:
from serpapi import GoogleSearch

relevant_urls = []

for term in search_terms:
    params = {
        "engine": "google",
        "q": term,
        "api_key": SERP_API_KEY,
        "tbs": "sbd:1", # sort by date
        "tbm": "nws", # news
        "location": "New York, New York, United States",
        "gl": "us",
        "hl": "en",
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    news_results = results["news_results"]

    news_results_title = "\n".join([f"{i+1}. {result['title']}" for i, result in enumerate(news_results)])

    system_prompt = "You are a journalist assistant. From the given search results, select the title that seems most relevant and informative for writing an article on the topic."
    messages = [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": f"Search Results:\n{news_results_title}\n\nPlease select the single number of the title that seems most relevant for writing an article on the topic '{topic}'. Respond with the single number of the title that seems most relevant. Ensure return the number only.\n\ne.g. 1"
        }]
    }]

    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=2,
        temperature=0,
        system=system_prompt,
        messages=messages,
    )
    relevant_id = int(message.content[0].text) - 1
    print(relevant_id)
    relevant_urls.extend([news_results[relevant_id]["link"]])

print(relevant_urls)

1
3
4
8
1
['https://www.sify.com/science-tech/iphone-16-price-release-date-specs-more/', 'https://www.indiatoday.in/amp/technology/news/story/iphone-16-vs-iphone-15-india-price-specs-features-and-more-compared-2598601-2024-09-12', 'https://www.macrumors.com/guide/iphone-15-pro-vs-iphone-16-pro/', 'https://www.businessinsider.com/guides/tech/iphone-16', 'https://wccftech.com/iphone-16-wired-charging-speeds-go-up-to-45w/']


In [69]:
news_results[0]

{'position': 1,
 'link': 'https://www.techradar.com/phones/iphone/iphone-16-pro-max',
 'title': 'iPhone 16 Pro Max: a new button, improved cameras, a bigger screen, and more',
 'source': 'TechRadar',
 'date': '1 hour ago',
 'snippet': 'The most highly anticipated smartphone series of the year has arrived, with \nthe iPhone 16, the iPhone 16 Plus, the iPhone 16 Pro, and – at the top of \nthe...',
 'thumbnail': 'https://serpapi.com/searches/66e2dd75987217328c5bb8d6/images/33f4ee0c0ec4acc9879e1f062231bc37ba2f6599b3c0e9f9b0fc7e1d9cc28393.jpeg'}

## Extract article content

각 기사의 본문 내용 크롤링 및 추출, 파싱 (Newspaper3k)

In [67]:
from newspaper import Article

article_texts = ""

for url in relevant_urls:
    try:
        article = Article(url)
        article.download()
        article.parse()
        article_texts += article.text
    except Exception as e:
        print(f"Error parsing article from {url}: {e}")

print(len(article_texts))
print(article_texts[:500])

42435
As the iPhone 16 release date gets announced, here’s all you need to know about Apple’s latest offering!

Apple introduced its latest iPhone earlier this month and at long last, we have all the details! The iPhone 16 and the iPhone 16 Plus will be releasing alongside the iPhone 16 Pro and the iPhone 16 Pro Max.

All four models will come with an upgraded A18 chip that supports Apple Intelligence, a dual lens camera that takes extremely, a Camera Control button to access the camera quickly and an


## Generate article

Claude 3.5 Sonnet 모델을 사용하여 고품질의 기사 생성

In [68]:
system_prompt = "You are a journalist. Write a high-quality, NYT-worthy article on the given topic based on the provided article texts. The article should be well-structured, informative, and engaging to Korean readers."
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": f"Topic: {topic}\n\nArticle Texts:\n{article_texts}\n\nPlease write a high-quality, NYT-worthy article on the topic based on the provided article texts. The article should be well-structured, informative, and engaging. Ensure the length is at least as long as a NYT cover story -- at a minimum, 15 paragraphs. Ensure the article is written in Korean."
    }]
}]

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=3000,
    temperature=0.5,
    system=system_prompt,
    messages=messages,
)

print(message.content[0].text)

애플의 새로운 아이폰 16 시리즈: AI와 혁신의 만남

애플이 지난 월요일 "Glowtime" 이벤트에서 아이폰 16 시리즈를 공식 발표했다. 이번 신제품은 애플의 새로운 AI 기술인 '애플 인텔리전스'를 탑재하고, 새로운 세대의 프로세서와 하드웨어 업그레이드를 포함하여 지금까지 출시된 아이폰 중 가장 뛰어난 성능을 자랑할 것으로 기대된다.

아이폰 16 시리즈는 기존 아이폰 15 시리즈와 동일하게 네 가지 모델로 구성된다: 아이폰 16, 아이폰 16 플러스, 아이폰 16 프로, 아이폰 16 프로 맥스. 이번에도 '미니' 모델은 출시되지 않았으며, 루머로 떠돌던 '울트라' 모델 역시 이번에는 등장하지 않았다.

새로운 아이폰 16 시리즈의 가장 큰 특징은 AI 기능인 '애플 인텔리전스'의 도입이다. 이 기능은 짧은 메모를 완전한 문장으로 바꾸거나, AI 이모지를 생성하고, 사진을 텍스트 설명으로 찾는 등 다양한 기능을 제공한다. 또한 시리의 성능도 크게 개선되어, 화면에 보이는 내용을 인식하고 그에 따른 작업을 수행할 수 있게 되었다.

카메라 부문에서도 상당한 개선이 이루어졌다. 기본 모델인 아이폰 16과 16 플러스는 전작과 동일한 48MP 메인 카메라와 12MP 초광각 카메라를 탑재했지만, 초광각 카메라에 오토포커스 기능이 추가되어 매크로 촬영이 가능해졌다. 프로 모델들은 더욱 강력한 카메라 시스템을 갖추었는데, 특히 아이폰 16 프로의 줌 카메라가 3배에서 5배로 업그레이드된 점이 눈에 띈다.

성능 면에서는 모든 모델이 새로운 A18 프로세서를 탑재했다. 애플은 기본 모델의 경우 전작 대비 30% 빠른 성능을, 프로 모델의 경우 AI 처리 속도가 15% 향상되었다고 밝혔다. 이는 게임 및 AI 애플리케이션 실행 시 더욱 빠르고 부드러운 경험을 제공할 것으로 기대된다.

디스플레이 부문에서는 프로 모델들의 화면 크기가 커졌다. 아이폰 16 프로는 6.3인치, 프로 맥스는 6.9인치로 각각 0.2인치씩 증가했다. 또한 모든 모델에 2세대 세라믹 실드 글

In [70]:
with open("news.txt", "w", encoding="utf-8") as f:
    f.write(message.content[0].text)